## Μηχανή Αναζήτησης (Wiki)

In [1]:
import requests
import json
import re
import os

from bs4 import BeautifulSoup

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from collections import defaultdict, Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi

Εισάγει τις απαραίτητες βιβλιοθήκες και μονάδες για την κατασκευή μιας μηχανής αναζήτησης άρθρων Wikipedia με διαφορετικά μοντέλα ανάκτησης (Boolean Retrieval, Vector Space Model (VSM), και Okapi BM25).

In [2]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

Η συνάρτηση ***preprocess_text*** έχει σχεδιαστεί για να καθαρίζει και να προετοιμάζει μη επεξεργασμένο κείμενο για εργασίες επεξεργασίας φυσικής γλώσσας (NLP). Ξεκινά με την αφαίρεση όλων των μη αλφαβητικών χαρακτήρων (όπως αριθμοί, σημεία στίξης και ειδικά σύμβολα) χρησιμοποιώντας μια κανονική έκφραση, αφήνοντας μόνο γράμματα και κενά. Στη συνέχεια, το κείμενο μετατρέπεται σε πεζά για να διασφαλιστεί η συνέπεια και γίνεται tokenized σε μεμονωμένες λέξεις. Οι κοινές stopwords (π.χ., "the," "is," "and") φιλτράρονται χρησιμοποιώντας τη βιβλιοθήκη NLTK, ώστε να διατηρηθούν μόνο οι ουσιαστικές λέξεις. Μετά από αυτό, η συνάρτηση εφαρμόζει stemming χρησιμοποιώντας τον αλγόριθμο Porter Stemming, μειώνοντας τις λέξεις στις ρίζες τους (π.χ., "running" γίνεται "run" και "cars" γίνεται "car"). Τέλος, τα επεξεργασμένα tokens επιστρέφονται ως λίστα, έτοιμη για περαιτέρω ανάλυση. Αυτή η προεπεξεργασία διασφαλίζει ότι το κείμενο είναι καθαρό, κανονικοποιημένο και κατάλληλο για εργασίες όπως αναζήτηση, ταξινόμηση ή μηχανική μάθηση.

In [3]:
def create_inverted_index(articles):
    inverted_index = defaultdict(list)
    
    for doc_id, article in enumerate(articles):
        tokens = article['content']
        term_freq = Counter(tokens)
        for token, freq in term_freq.items():
            inverted_index[token].append({"doc_id": doc_id, "frequency": freq})

    return inverted_index

Η συνάρτηση ***create_inverted_index*** κατασκευάζει ένα inverted index από μια λίστα άρθρων. Ένα inverted index είναι μια δομή δεδομένων που χρησιμοποιείται συνήθως σε συστήματα ανάκτησης πληροφοριών για να αντιστοιχίσει κάθε μοναδικό token (ή όρο) στη λίστα των εγγράφων στα οποία εμφανίζεται, μαζί με τη συχνότητά του σε κάθε έγγραφο. Η συνάρτηση αρχικοποιεί ένα κενό defaultdict για την αποθήκευση του index. Για κάθε άρθρο, ανακτά τα προεπεξεργασμένα tokens από το πεδίο content και υπολογίζει τις term frequencies χρησιμοποιώντας την κλάση Counter. Στη συνέχεια, για κάθε token, προσθέτει μια εγγραφή στο inverted index που περιλαμβάνει το document ID (doc_id) και τη συχνότητα του token μέσα στο έγγραφο. Η προκύπτουσα δομή επιτρέπει αποδοτικές αναζητήσεις για το ποια έγγραφα περιέχουν συγκεκριμένα tokens και τις αντίστοιχες συχνότητές τους, καθιστώντας την έναν κρίσιμο παράγοντα για υλοποιήσεις μηχανών αναζήτησης.

In [4]:
def fetch_wikipedia_articles(seed_url, max_articles):
    visited_urls = set()
    articles = []
    urls_to_visit = [seed_url]

    while urls_to_visit and len(articles) < max_articles:
        current_url = urls_to_visit.pop(0)
        if current_url in visited_urls:
            continue

        response = requests.get(current_url)
        if response.status_code != 200:
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.find('h1', {'id': 'firstHeading'}).text
        content = ''
        for paragraph in soup.find_all('p'):
            content += paragraph.text

        processed_content = preprocess_text(content)
        articles.append({"title": title, "url": current_url, "content": processed_content})
        visited_urls.add(current_url)

        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.startswith("/wiki/") and not any(x in href for x in [":", "#"]):
                full_url = f"https://en.wikipedia.org{href}"
                if full_url not in visited_urls and full_url not in urls_to_visit:
                    urls_to_visit.append(full_url)

    return articles

Η συνάρτηση ***fetch_wikipedia_articles*** ανακτά και επεξεργάζεται άρθρα από τη Wikipedia, ξεκινώντας από ένα συγκεκριμένο seed URL. Εκτελεί μια ευρέως πρώτα διάσχιση (breadth-first traversal) των συνδέσμων της Wikipedia για να συλλέξει έναν καθορισμένο μέγιστο αριθμό άρθρων (max_articles). Η συνάρτηση διατηρεί ένα σύνολο visited_urls για να αποφύγει την επεξεργασία της ίδιας σελίδας πολλαπλές φορές και μια ουρά urls_to_visit για τη διαχείριση των συνδέσμων που δεν έχουν ακόμα εξερευνηθεί.

Για κάθε URL, εκτελεί ένα HTTP request χρησιμοποιώντας τη βιβλιοθήκη requests. Εάν η απόκριση είναι επιτυχής, το HTML περιεχόμενο αναλύεται με το BeautifulSoup για να εξαχθούν ο τίτλος της σελίδας και το κείμενο από τα < p > tags, που αντιπροσωπεύουν το κύριο περιεχόμενο. Αυτό το κείμενο στη συνέχεια υφίσταται προεπεξεργασία μέσω της συνάρτησης preprocess_text, και το επεξεργασμένο περιεχόμενο αποθηκεύεται μαζί με τον τίτλο και το URL σε μια λίστα άρθρων.

Επιπλέον, η συνάρτηση εντοπίζει όλους τους εσωτερικούς συνδέσμους της Wikipedia (/wiki/) από την τρέχουσα σελίδα, εξαιρώντας αυτούς που περιέχουν ειδικούς χαρακτήρες όπως : ή #, και τους προσθέτει στην ουρά αν δεν έχουν ήδη επισκεφθεί ή προστεθεί. Μόλις συγκεντρωθεί ο επιθυμητός αριθμός άρθρων ή δεν υπάρχουν άλλοι σύνδεσμοι προς εξερεύνηση, η συνάρτηση επιστρέφει μια λίστα από dictionaries που περιέχουν τον τίτλο, το URL, και το επεξεργασμένο περιεχόμενο για κάθε άρθρο. Αυτή η συνάρτηση είναι ουσιώδης για την κατασκευή ενός dataset άρθρων της Wikipedia.

In [5]:
def boolean_retrieval(query, inverted_index):
    tokens = word_tokenize(query.lower())
    and_results = None
    or_results = set()
    not_results = set()
    prev_token = None
    current_set = None

    for token in tokens:
        if token == 'and':
            prev_token = 'AND'
            if current_set is not None:
                if and_results is None:
                    and_results = current_set
                else:
                    and_results &= current_set
            current_set = None
        elif token == 'or':
            prev_token = 'OR'
            if current_set is not None:
                or_results |= current_set
            current_set = None
        elif token == 'not':
            prev_token = 'NOT'
            continue
        else:
            if token in inverted_index:
                current_set = set(doc['doc_id'] for doc in inverted_index[token])
            else:
                current_set = set()

            if prev_token == 'NOT':
                not_results |= current_set
                current_set = set()
            elif prev_token == 'AND':
                if and_results is None:
                    and_results = current_set
                else:
                    and_results &= current_set
            elif prev_token == 'OR':
                or_results |= current_set
            else:
                if and_results is None:
                    and_results = current_set

    if and_results is not None:
        and_results -= not_results

    return (and_results or set()) | (or_results - not_results)

Η συνάρτηση ***boolean_retrieval*** εκτελεί αναζήτηση Boolean query σε ένα δεδομένο inverted index. Η Boolean ανάκτηση επιτρέπει στους χρήστες να συνδυάζουν όρους αναζήτησης χρησιμοποιώντας λογικούς τελεστές όπως AND, OR και NOT. Η συνάρτηση λαμβάνει ένα query του χρήστη ως string και ένα inverted_index ως είσοδο, όπου το inverted index αντιστοιχεί tokens στα έγγραφα στα οποία εμφανίζονται.

**Tokenization:** Το query μετατρέπεται σε πεζά και tokenized χρησιμοποιώντας τη συνάρτηση word_tokenize.

**Λογική Επεξεργασία:**

AND: Τα αποτελέσματα πρέπει να περιλαμβάνουν όλους τους όρους που συνδυάζονται με AND.

OR: Τα αποτελέσματα μπορούν να περιλαμβάνουν οποιονδήποτε όρο που συνδυάζεται με OR.

NOT: Τα αποτελέσματα πρέπει να εξαιρούν έγγραφα που περιέχουν όρους μετά από NOT.

**Σύνολα Αποτελεσμάτων:**

Το and_results αποθηκεύει έγγραφα που πληρούν τα κριτήρια για το AND.

Το or_results συσσωρεύει έγγραφα που πληρούν τα κριτήρια για το OR.

Το not_results αποθηκεύει έγγραφα που πρέπει να εξαιρεθούν.

Η μεταβλητή current_set περιέχει έγγραφα που αντιστοιχούν στο τρέχον token. Καθώς τα query tokens επεξεργάζονται:

Αν ένα token είναι λογικός τελεστής (AND, OR, NOT), ενημερώνει την κατάσταση επεξεργασίας.
Αν ένα token είναι όρος αναζήτησης, η συνάρτηση το αναζητά στο inverted index για να ανακτήσει το σύνολο των αντίστοιχων document IDs. Αυτό το σύνολο συνδυάζεται με τα προηγούμενα αποτελέσματα βάσει του τρέχοντος λογικού τελεστή.
Μετά την επεξεργασία όλων των tokens, η συνάρτηση υπολογίζει το τελικό αποτέλεσμα συνδυάζοντας τα and_results, or_results και εξαιρώντας τα έγγραφα στο not_results. Η συνάρτηση επιστρέφει το σύνολο των document IDs που αντιστοιχούν στο Boolean query.

Αυτή η προσέγγιση υποστηρίζει ευέλικτη και αποδοτική ανάκτηση εγγράφων με βάση λογικές εκφράσεις που ορίζει ο χρήστης.

In [6]:
def compute_tfidf(articles):
    texts = [" ".join(article['content']) for article in articles]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    return vectorizer, tfidf_matrix

Η συνάρτηση ***compute_tfidf*** υπολογίζει την αναπαράσταση Term Frequency-Inverse Document Frequency (TF-IDF) για ένα σύνολο άρθρων. Το TF-IDF είναι ένα στατιστικό μέτρο που χρησιμοποιείται για την αξιολόγηση της σημασίας των λέξεων σε ένα έγγραφο σε σχέση με μια συλλογή (ή corpus) εγγράφων.

**Είσοδος:** Η συνάρτηση λαμβάνει μια λίστα άρθρων, όπου κάθε άρθρο είναι ένα dictionary που περιέχει ένα πεδίο content (μια λίστα από προεπεξεργασμένα tokens).

**Προετοιμασία Κειμένου:** Τα tokens σε κάθε άρθρο ενώνονται σε ένα ενιαίο string χρησιμοποιώντας " ".join(article['content']), δημιουργώντας μια λίστα εγγράφων που αναπαρίστανται ως απλό κείμενο.

**TF-IDF Vectorizer:** Ένας TfidfVectorizer από το sklearn.feature_extraction.text αρχικοποιείται. Αυτό το εργαλείο μετατρέπει τα δεδομένα κειμένου σε έναν TF-IDF πίνακα, όπου οι γραμμές αντιπροσωπεύουν έγγραφα και οι στήλες αντιπροσωπεύουν όρους.

**Κατασκευή Πίνακα:** Η μέθοδος fit_transform του vectorizer εφαρμόζεται στη λίστα εγγράφων, παράγοντας έναν sparse matrix (tfidf_matrix), όπου:

Κάθε καταχώρηση αντιπροσωπεύει τη βαθμολογία TF-IDF ενός όρου σε ένα έγγραφο.
Υψηλότερες βαθμολογίες υποδεικνύουν όρους που είναι πιο μοναδικοί σε ένα συγκεκριμένο έγγραφο.

**Έξοδος:** Η συνάρτηση επιστρέφει τον vectorizer (χρήσιμο για τη μετατροπή μελλοντικών queries) και τον tfidf_matrix, που χρησιμοποιείται για εργασίες όπως ο υπολογισμός ομοιότητας.

Αυτή η συνάρτηση αποτελεί κρίσιμο βήμα για την ενεργοποίηση του vector space modeling, όπου τα έγγραφα και τα queries αναπαρίστανται σε έναν πολυδιάστατο χώρο για αποδοτική κατάταξη και ανάκτηση.

In [7]:
def vsm_ranking(query, vectorizer, tfidf_matrix, threshold = 0.1):
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    ranked_docs = [
        doc_id for doc_id, score in enumerate(cosine_similarities) if score >= threshold
    ]
    ranked_docs = sorted(ranked_docs, key = lambda doc_id: cosine_similarities[doc_id], reverse = True)
    return ranked_docs

Η συνάρτηση ***vsm_ranking*** κατατάσσει έγγραφα με βάση τη συνάφειά τους με ένα δεδομένο query χρησιμοποιώντας το Vector Space Model (VSM) και την cosine similarity. Αυτή η μέθοδος χρησιμοποιείται ευρέως σε συστήματα ανάκτησης πληροφοριών για την εύρεση των πιο σχετικών εγγράφων για ένα query αναζήτησης.

**Είσοδοι:**
**query:** Το query αναζήτησης ως string.
**vectorizer:** Ένα προ-εκπαιδευμένο αντικείμενο TfidfVectorizer για τη μετατροπή του query σε TF-IDF vector.
**tfidf_matrix:** Ένας TF-IDF πίνακας που αναπαριστά τα προεπεξεργασμένα άρθρα.
**threshold (προεπιλογή: 0.1):** Η ελάχιστη βαθμολογία cosine similarity που απαιτείται για να θεωρηθεί ένα έγγραφο σχετικό.

**Μετατροπή Query:** Το query μετατρέπεται σε TF-IDF vector χρησιμοποιώντας τη μέθοδο vectorizer.transform. Αυτό διασφαλίζει ότι το query αναπαρίσταται στον ίδιο χώρο χαρακτηριστικών με τα document vectors.

**Υπολογισμός Cosine Similarity:** Η cosine similarity μεταξύ του query vector και κάθε document vector στον tfidf_matrix υπολογίζεται χρησιμοποιώντας τη συνάρτηση cosine_similarity. Το αποτέλεσμα είναι μια λίστα βαθμολογιών ομοιότητας, όπου κάθε βαθμολογία δείχνει πόσο κοντά ταιριάζει το έγγραφο με το query.

**Φιλτράρισμα με Threshold:** Έγγραφα με βαθμολογία ομοιότητας μεγαλύτερη ή ίση με το threshold θεωρούνται σχετικά. Οι δείκτες τους (doc_id) αποθηκεύονται στη λίστα ranked_docs.

**Κατάταξη:** Τα σχετικά έγγραφα ταξινομούνται με φθίνουσα σειρά βαθμολογιών ομοιότητας, διασφαλίζοντας ότι τα πιο σχετικά έγγραφα εμφανίζονται πρώτα.

**Έξοδος:** Η συνάρτηση επιστρέφει τη λίστα ranked_docs, μια λίστα δεικτών εγγράφων ταξινομημένων κατά συνάφεια.

Αυτή η συνάρτηση είναι κρίσιμη για την ενεργοποίηση κατάταξης ανάκτησης πληροφοριών σε έναν πολυδιάστατο χώρο, όπου η ομοιότητα μεταξύ εγγράφων και queries χρησιμοποιείται για τον καθορισμό της συνάφειας.

In [8]:
def okapi_bm25(query, articles, threshold = 0.1):
    tokenized_corpus = [article['content'] for article in articles]
    bm25 = BM25Okapi(tokenized_corpus)
    query_tokens = preprocess_text(query)
    scores = bm25.get_scores(query_tokens)
    ranked_docs = [
        doc_id for doc_id, score in enumerate(scores) if score >= threshold
    ]
    ranked_docs = sorted(ranked_docs, key = lambda doc_id: scores[doc_id], reverse = True)
    return ranked_docs

Η συνάρτηση ***okapi_bm25*** κατατάσσει έγγραφα με βάση τη συνάφειά τους με ένα query χρησιμοποιώντας τον αλγόριθμο Okapi BM25, ένα δημοφιλές probabilistic information retrieval μοντέλο που αξιολογεί τη συνάφεια εγγράφου-query.

**Είσοδοι:**
**query:** Το query αναζήτησης ως string.
**articles:** Μια λίστα από προεπεξεργασμένα άρθρα, όπου κάθε άρθρο περιέχει tokenized περιεχόμενο.
**threshold (προεπιλογή: 0.1):** Η ελάχιστη βαθμολογία BM25 που πρέπει να επιτύχει ένα έγγραφο για να θεωρηθεί σχετικό.

**Προετοιμασία Corpus:** Το tokenized_corpus δημιουργείται με την εξαγωγή του tokenized περιεχομένου κάθε άρθρου. Αυτό αναπαριστά τη συλλογή εγγράφων.

**Αρχικοποίηση BM25:** Ένα αντικείμενο BM25Okapi αρχικοποιείται χρησιμοποιώντας το tokenized_corpus. Αυτό προϋπολογίζει τις απαραίτητες στατιστικές (π.χ., term frequencies και μήκη εγγράφων) για αποδοτική βαθμολόγηση.

**Προεπεξεργασία Query:** Το query προεπεξεργάζεται χρησιμοποιώντας τη συνάρτηση preprocess_text, η οποία αφαιρεί τα stopwords, μετατρέπει το κείμενο σε πεζά και εφαρμόζει stemming. Αυτό διασφαλίζει τη συνέπεια μεταξύ του query και του tokenized corpus.

**Υπολογισμός Βαθμολογιών:** Ο αλγόριθμος BM25 υπολογίζει τις βαθμολογίες συνάφειας για κάθε έγγραφο στο corpus σε σχέση με το query. Το αποτέλεσμα είναι μια λίστα scores, όπου κάθε τιμή αντιπροσωπεύει τη βαθμολογία συνάφειας ενός εγγράφου.

**Φιλτράρισμα με Threshold:** Έγγραφα με βαθμολογίες μεγαλύτερες ή ίσες με το threshold θεωρούνται σχετικά. Οι δείκτες τους (doc_id) προστίθενται στη λίστα ranked_docs.

**Κατάταξη:** Τα σχετικά έγγραφα ταξινομούνται με φθίνουσα σειρά βάσει των βαθμολογιών BM25, διασφαλίζοντας ότι τα πιο σχετικά έγγραφα εμφανίζονται πρώτα.

**Έξοδος:** Η συνάρτηση επιστρέφει τη λίστα ranked_docs, μια λίστα δεικτών εγγράφων ταξινομημένων κατά συνάφεια σύμφωνα με τον αλγόριθμο BM25.

Αυτή η συνάρτηση προσφέρει έναν αποδοτικό και αξιόπιστο τρόπο κατάταξης εγγράφων με βάση τη συνάφεια του query, αξιοποιώντας τα πλεονεκτήματα του BM25 στην αντιμετώπιση της συχνότητας όρων, της κανονικοποίησης μήκους εγγράφων και της inverse document frequency.

In [9]:
def save_to_json(data, filename = "wikipedia_index.json"):
    with open(filename, 'w', encoding = 'utf-8') as f:
        json.dump(data, f, ensure_ascii = False, indent = 4)

Η συνάρτηση ***save_to_json*** έχει σχεδιαστεί για να αποθηκεύει δεδομένα σε ένα αρχείο JSON. Δέχεται δύο ορίσματα: data, που αναπαριστά το Python αντικείμενο (όπως dictionary ή list) που θέλετε να αποθηκεύσετε, και filename (που έχει προεπιλεγμένη τιμή "wikipedia_index.json"), το όνομα του αρχείου όπου θα αποθηκευτούν τα δεδομένα. Η συνάρτηση ανοίγει το αρχείο σε λειτουργία εγγραφής ('w') χρησιμοποιώντας κωδικοποίηση UTF-8, διασφαλίζοντας ότι οι μη-ASCII χαρακτήρες αντιμετωπίζονται σωστά. Η μέθοδος json.dump χρησιμοποιείται για να σειριοποιήσει τα δεδομένα και να τα γράψει στο αρχείο.

Η παράμετρος ensure_ascii=False διασφαλίζει ότι οποιοιδήποτε μη-ASCII χαρακτήρες διατηρούνται στην αρχική τους μορφή, καθιστώντας το JSON αρχείο πιο ευανάγνωστο για περιεχόμενο πολλών γλωσσών. Επιπλέον, η παράμετρος indent=4 μορφοποιεί τα δεδομένα JSON με εσοχή 4 κενών, διευκολύνοντας την ανάγνωση και κατανόησή τους.

Αυτή η συνάρτηση είναι ιδιαίτερα χρήσιμη για την αποθήκευση δομημένων δεδομένων, όπως τα αποτελέσματα μιας διαδικασίας ευρετηρίασης ή επεξεργασμένου περιεχομένου, έτσι ώστε να μπορούν να φορτωθούν και να επαναχρησιμοποιηθούν αργότερα.

In [10]:
def load_data():
    with open("wikipedia_index.json", "r", encoding = 'utf-8') as f:
        data = json.load(f)
    return data['articles'], create_inverted_index(data['articles'])

Η συνάρτηση ***load_data*** είναι υπεύθυνη για τη φόρτωση δεδομένων που έχουν αποθηκευτεί προηγουμένως από ένα αρχείο JSON, συγκεκριμένα το αρχείο "wikipedia_index.json". Ανοίγει το αρχείο σε λειτουργία ανάγνωσης ('r') με κωδικοποίηση UTF-8, διασφαλίζοντας ότι οποιοιδήποτε μη-ASCII χαρακτήρες αντιμετωπίζονται σωστά. Στη συνέχεια, η συνάρτηση χρησιμοποιεί τη μέθοδο json.load για να αναλύσει τα περιεχόμενα του αρχείου και να μετατρέψει τα δεδομένα JSON σε ένα Python dictionary.

Αφού φορτωθούν τα δεδομένα, η συνάρτηση εξάγει τη λίστα των άρθρων από το dictionary χρησιμοποιώντας data['articles'] και περνά αυτή τη λίστα στη συνάρτηση create_inverted_index για να δημιουργήσει το αντίστοιχο inverted index. Τέλος, επιστρέφει δύο αντικείμενα: τη λίστα των άρθρων και το inverted index, τα οποία είναι απαραίτητα για τη λειτουργικότητα της μηχανής αναζήτησης.

Αυτή η συνάρτηση είναι κρίσιμη για τη φόρτωση και την προετοιμασία των δεδομένων για περαιτέρω επεξεργασία ή για queries αναζήτησης.

In [11]:
def load_test_queries(filename):
    with open(filename, "r", encoding = 'utf-8') as f:
        return json.load(f)

Η συνάρτηση ***load_test_queries*** χρησιμοποιείται για τη φόρτωση test queries από ένα συγκεκριμένο αρχείο JSON. Δέχεται ένα όρισμα, filename, που είναι το όνομα του αρχείου που περιέχει τα test queries. Η συνάρτηση ανοίγει το αρχείο σε λειτουργία ανάγνωσης ('r') χρησιμοποιώντας κωδικοποίηση UTF-8, για να διαχειριστεί σωστά τυχόν ειδικούς χαρακτήρες. Στη συνέχεια, χρησιμοποιεί τη μέθοδο json.load για να αναλύσει το περιεχόμενο του JSON αρχείου και να το μετατρέψει σε ένα Python αντικείμενο, συνήθως ένα dictionary ή μια λίστα.

Η συνάρτηση επιστρέφει τα φορτωμένα δεδομένα, τα οποία συνήθως αποτελούνται από μια λίστα queries που μπορούν να χρησιμοποιηθούν για την αξιολόγηση της απόδοσης μιας μηχανής αναζήτησης ή ενός συστήματος ανάκτησης πληροφορίας.

Αυτή η συνάρτηση είναι χρήσιμη για τη φόρτωση προκαθορισμένων queries, με σκοπό το benchmarking ή τη δοκιμή της αποτελεσματικότητας αλγορίθμων αναζήτησης όπως Boolean retrieval, VSM ή BM25.

In [12]:
def precision(retrieved, relevant):
    if len(retrieved) == 0:
        return 0.0
    return len(set(retrieved) & set(relevant)) / len(retrieved)

Η συνάρτηση ***precision*** υπολογίζει την ακρίβεια ενός αποτελέσματος αναζήτησης, η οποία μετρά το ποσοστό των σχετικών εγγράφων που ανακτήθηκαν επιτυχώς. Δέχεται δύο ορίσματα: retrieved, μια λίστα με τα IDs ή τις αναφορές των εγγράφων που επέστρεψε η μηχανή αναζήτησης, και relevant, μια λίστα με τα IDs ή τις αναφορές των εγγράφων που θεωρούνται σχετικά για το συγκεκριμένο query.

Η συνάρτηση πρώτα ελέγχει αν η λίστα retrieved είναι κενή. Αν είναι, η ακρίβεια ορίζεται σε 0.0, επειδή δεν ανακτήθηκαν έγγραφα. Αν έχουν ανακτηθεί έγγραφα, η συνάρτηση υπολογίζει την ακρίβεια βρίσκοντας την τομή (intersection) των συνόλων retrieved και relevant (δηλαδή τον αριθμό των σχετικών εγγράφων που πράγματι ανακτήθηκαν). Αυτό γίνεται χρησιμοποιώντας τον τελεστή & μεταξύ των συνόλων. Το αποτέλεσμα στη συνέχεια διαιρείται με τον συνολικό αριθμό των εγγράφων στη λίστα retrieved, για να υπολογιστεί η ακρίβεια. Όσο μεγαλύτερη είναι η ακρίβεια, τόσο καλύτερη είναι η ικανότητα της μηχανής αναζήτησης να ανακτά σχετικά έγγραφα και να φιλτράρει τα μη σχετικά.

In [13]:
def recall(retrieved, relevant):
    if len(relevant) == 0:
        return 0.0
    return len(set(retrieved) & set(relevant)) / len(relevant)

Η συνάρτηση ***recall*** υπολογίζει την ανάκληση ενός αποτελέσματος αναζήτησης, η οποία μετρά το ποσοστό των σχετικών εγγράφων που ανακτήθηκαν επιτυχώς. Δέχεται δύο ορίσματα: retrieved, μια λίστα με τα IDs ή τις αναφορές των εγγράφων που επέστρεψε η μηχανή αναζήτησης, και relevant, μια λίστα με τα IDs ή τις αναφορές των εγγράφων που θεωρούνται σχετικά για το συγκεκριμένο query.

Η συνάρτηση πρώτα ελέγχει αν η λίστα relevant είναι κενή. Αν είναι, η ανάκληση ορίζεται σε 0.0, καθώς δεν υπάρχουν σχετικά έγγραφα προς ανάκτηση. Αν υπάρχουν σχετικά έγγραφα, η συνάρτηση υπολογίζει την ανάκληση βρίσκοντας την τομή (intersection) των συνόλων retrieved και relevant (δηλαδή τον αριθμό των σχετικών εγγράφων που πράγματι ανακτήθηκαν). Αυτό γίνεται χρησιμοποιώντας τον τελεστή & μεταξύ των συνόλων. Το αποτέλεσμα στη συνέχεια διαιρείται με τον συνολικό αριθμό των σχετικών εγγράφων, για να υπολογιστεί η ανάκληση. Μια υψηλότερη ανάκληση υποδεικνύει ότι η μηχανή αναζήτησης έχει ανακτήσει μεγαλύτερο ποσοστό των σχετικών εγγράφων, αλλά μπορεί επίσης να ανακτήσει περισσότερα μη σχετικά έγγραφα.

In [14]:
def f1_score(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

Η συνάρτηση ***f1_score*** υπολογίζει το F1 score, το οποίο είναι ο αρμονικός μέσος της precision και της recall. Το F1 score παρέχει μια ισορροπία μεταξύ της precision και της recall, προσφέροντας έναν ενιαίο δείκτη που συνδυάζει και τις δύο πτυχές της απόδοσης μιας μηχανής αναζήτησης. Είναι ιδιαίτερα χρήσιμο όταν θέλετε να διασφαλίσετε ότι τόσο η precision (ανάκτηση σχετικών εγγράφων) όσο και η recall (ανάκτηση όλων των σχετικών εγγράφων) είναι ισορροπημένες.

Η συνάρτηση δέχεται δύο ορίσματα: precision και recall, που είναι τα αποτελέσματα από τους αντίστοιχους υπολογισμούς της precision και της recall.

Αν και οι δύο τιμές, precision και recall, είναι μηδέν (που σημαίνει ότι δεν ανακτήθηκαν σχετικά έγγραφα και δεν υπάρχουν σχετικά έγγραφα), η συνάρτηση επιστρέφει ένα F1 score ίσο με 0.0. Διαφορετικά, υπολογίζει το F1 score χρησιμοποιώντας τον τύπο:

### F1 = 2 × (precision × recall) / (precision + recall)

Το F1 score κυμαίνεται μεταξύ 0 (κακή απόδοση) και 1 (τέλεια απόδοση). Ένα υψηλότερο F1 score υποδεικνύει μια καλύτερη συνολική ισορροπία μεταξύ της precision και της recall.

In [15]:
def average_precision(retrieved, relevant):
    relevant_retrieved = 0
    precision_sum = 0
    for i, doc_id in enumerate(retrieved):
        if doc_id in relevant:
            relevant_retrieved += 1
            precision_sum += relevant_retrieved / (i + 1)
    if relevant_retrieved == 0:
        return 0.0
    return precision_sum / relevant_retrieved

Η συνάρτηση ***average_precision*** υπολογίζει την μέση ακρίβεια ενός αποτελέσματος αναζήτησης με βάση τη σχετικότητα των ανακτημένων εγγράφων. Δέχεται δύο ορίσματα: retrieved (μια λίστα με τα IDs των εγγράφων που ανακτήθηκαν από τον αλγόριθμο αναζήτησης) και relevant (μια λίστα με τα IDs των σχετικών εγγράφων για το query).

Η συνάρτηση ξεκινά με την αρχικοποίηση δύο μεταβλητών: relevant_retrieved (για την παρακολούθηση του αριθμού των σχετικών εγγράφων που ανακτήθηκαν) και precision_sum (για τη συσσώρευση της precision σε κάθε θέση όπου ανακτάται ένα σχετικό έγγραφο). Στη συνέχεια, διατρέχει τη λίστα των ανακτημένων εγγράφων, ελέγχοντας αν κάθε έγγραφο είναι σχετικό με τον έλεγχο της παρουσίας του στη λίστα relevant. Αν βρεθεί ένα σχετικό έγγραφο, ενημερώνει τον μετρητή relevant_retrieved και προσθέτει την precision σε αυτήν τη θέση στη precision_sum. Η precision σε κάθε θέση υπολογίζεται ως η αναλογία των σχετικών εγγράφων που ανακτήθηκαν μέχρι εκείνη τη στιγμή διαιρεμένη με την τρέχουσα θέση.

Μετά την επεξεργασία όλων των ανακτημένων εγγράφων, αν δεν ανακτήθηκαν σχετικά έγγραφα, η συνάρτηση επιστρέφει 0.0, υποδεικνύοντας κακή ανάκτηση. Διαφορετικά, επιστρέφει τη μέση ακρίβεια, η οποία είναι το άθροισμα των τιμών precision διαιρεμένο με τον συνολικό αριθμό των σχετικών εγγράφων που ανακτήθηκαν. Αυτός ο δείκτης παρέχει μια ένδειξη της ικανότητας της μηχανής αναζήτησης να κατατάξει τα σχετικά έγγραφα ψηλά στα αποτελέσματα. Μια υψηλότερη μέση ακρίβεια υποδεικνύει ότι τα πιο σχετικά έγγραφα εμφανίζονται νωρίτερα στα αποτελέσματα αναζήτησης.

In [16]:
def mean_average_precision(queries, algorithm_results):
    map_score = 0
    for query_data in queries['queries']:
        query = query_data['query']
        relevant_docs = query_data['relevant_docs']
        
        retrieved_boolean = algorithm_results['Boolean'][query]
        retrieved_vsm = algorithm_results['VSM'][query]
        retrieved_bm25 = algorithm_results['BM25'][query]
        
        ap_boolean = average_precision(retrieved_boolean, relevant_docs)
        ap_vsm = average_precision(retrieved_vsm, relevant_docs)
        ap_bm25 = average_precision(retrieved_bm25, relevant_docs)
        
        map_score += (ap_boolean + ap_vsm + ap_bm25) / 3
    
    return map_score / len(queries)

Η συνάρτηση ***mean_average_precision*** υπολογίζει την μέση μέση ακρίβεια (MAP) για ένα σύνολο queries με βάση τα αποτελέσματα από τρεις διαφορετικούς αλγόριθμους αναζήτησης: Boolean, VSM (Vector Space Model) και BM25. Δέχεται δύο ορίσματα: queries, που περιέχει τα queries και τα σχετικά έγγραφα τους, και algorithm_results, που περιέχει τα αποτελέσματα αναζήτησης για κάθε αλγόριθμο.

Για κάθε query στο queries, η συνάρτηση εξάγει το query string και τα σχετικά έγγραφα του. Στη συνέχεια, ανακτά τα IDs των εγγράφων που επιστράφηκαν από κάθε έναν από τους τρεις αλγόριθμους για εκείνο το query. Για κάθε σύνολο ανακτημένων εγγράφων (από Boolean, VSM και BM25), υπολογίζει την μέση ακρίβεια χρησιμοποιώντας τη συνάρτηση average_precision, η οποία μετράει πόσο καλά κατατάσσει ο αλγόριθμος τα σχετικά έγγραφα.

Οι βαθμολογίες μέσης ακρίβειας για τους τρεις αλγόριθμους αθροίζονται και αυτή η τιμή συσσωρεύεται σε όλα τα queries. Τελικά, η συνάρτηση επιστρέφει τον μέσο όρο αυτών των βαθμολογιών μέσης ακρίβειας, παρέχοντας έναν ενιαίο δείκτη που αντανακλά την συνολική απόδοση όλων των αλγορίθμων σε όλο το σύνολο των queries.

Το MAP είναι ένας χρήσιμος δείκτης για την αξιολόγηση των συστημάτων αναζήτησης, καθώς παρέχει έναν συνολικό δείκτη για το πόσο καλά το σύστημα κατατάσσει τα σχετικά έγγραφα, με υψηλότερο MAP να υποδεικνύει καλύτερη απόδοση.

In [17]:
def evaluate_search_algorithms(articles, inverted_index, vectorizer, tfidf_matrix):
    queries = load_test_queries("test_queries.json")

    algorithm_results = {
        'Boolean': {},
        'VSM': {},
        'BM25': {}
    }

    for query_data in queries['queries']:
        query = query_data['query']
        relevant_docs = query_data['relevant_docs']
        
        retrieved_boolean = boolean_retrieval(query, inverted_index)
        retrieved_vsm = vsm_ranking(query, vectorizer, tfidf_matrix, threshold = 0.2)
        retrieved_bm25 = okapi_bm25(query, articles, threshold = 1.0)
        
        algorithm_results['Boolean'][query] = retrieved_boolean
        algorithm_results['VSM'][query] = retrieved_vsm
        algorithm_results['BM25'][query] = retrieved_bm25
        
        precision_boolean = precision(retrieved_boolean, relevant_docs)
        recall_boolean = recall(retrieved_boolean, relevant_docs)
        f1_boolean = f1_score(precision_boolean, recall_boolean)

        precision_vsm = precision(retrieved_vsm, relevant_docs)
        recall_vsm = recall(retrieved_vsm, relevant_docs)
        f1_vsm = f1_score(precision_vsm, recall_vsm)

        precision_bm25 = precision(retrieved_bm25, relevant_docs)
        recall_bm25 = recall(retrieved_bm25, relevant_docs)
        f1_bm25 = f1_score(precision_bm25, recall_bm25)
        
        print(f"\n")
        print(f"Query: {query}")
        print(f"Boolean - Precision: {precision_boolean:.4f}, Recall: {recall_boolean:.4f}, F1: {f1_boolean:.4f}")
        print(f"VSM - Precision: {precision_vsm:.4f}, Recall: {recall_vsm:.4f}, F1: {f1_vsm:.4f}")
        print(f"BM25 - Precision: {precision_bm25:.4f}, Recall: {recall_bm25:.4f}, F1: {f1_bm25:.4f}")
    
    map_score = mean_average_precision(queries, algorithm_results)
    print(f"\n")
    print(f"Mean Average Precision (MAP): {map_score:.4f}")

Η συνάρτηση ***evaluate_search_algorithms*** έχει σχεδιαστεί για να αξιολογεί την απόδοση τριών αλγορίθμων αναζήτησης (Boolean, VSM και BM25) με βάση ένα σύνολο test queries. Η συνάρτηση δέχεται τέσσερα ορίσματα: articles (μια λίστα με άρθρα για αναζήτηση), inverted_index (ένας δείκτης που αντιστοιχίζει τους όρους στα έγγραφα), vectorizer (χρησιμοποιείται για τη μετατροπή κειμένων σε διανύσματα για το VSM), και tfidf_matrix (ο πίνακας που αναπαριστά τη συχνότητα όρου-αντίστροφη συχνότητα εγγράφου για τα άρθρα).

Η συνάρτηση ξεκινά με τη φόρτωση των test queries από ένα αρχείο JSON (test_queries.json), το οποίο περιέχει μια λίστα με queries και τα σχετικά έγγραφά τους. Αρχικοποιεί ένα λεξικό για να αποθηκεύσει τα αποτελέσματα αναζήτησης από κάθε αλγόριθμο για κάθε query.

**Για κάθε query, η συνάρτηση:**

Ανακτά τα σχετικά έγγραφα χρησιμοποιώντας τους τρεις αλγόριθμους: Boolean retrieval (boolean_retrieval), Vector Space Model (vsm_ranking), και BM25 (okapi_bm25).
Υπολογίζει την ακρίβεια, την ανάκληση και το F1 score για κάθε αλγόριθμο χρησιμοποιώντας τα αντίστοιχα αποτελέσματα αναζήτησης και τα σχετικά έγγραφα.
Εκτυπώνει τα αποτελέσματα ακρίβειας, ανάκλησης και F1 score για κάθε αλγόριθμο και query.
Αφού επεξεργαστεί όλα τα queries, η συνάρτηση υπολογίζει το σκορ Μέσης Μέσης Ακρίβειας (MAP) για όλα τα queries χρησιμοποιώντας τη συνάρτηση mean_average_precision και το εκτυπώνει.

Αυτή η συνάρτηση είναι χρήσιμη για τη σύγκριση της αποτελεσματικότητας των τριών αλγορίθμων αναζήτησης και την κατανόηση της απόδοσής τους σε όρους ακρίβειας, ανάκλησης, F1 score και συνολικής ποιότητας αναζήτησης (MAP).

In [18]:
def main():
    print(f"###Wikipedia Article Search Engine###")

    articles, inverted_index = load_data()

    vectorizer, tfidf_matrix = compute_tfidf(articles)

    while True:
        print(f"\n**Do you want to search a query or to evaluate the algorithms?**")
        print(f"1. Search")
        print(f"2. Evaluation")
        print(f"3. Exit\n")
        choice = int(input())

        if choice == 1:
            print(f"**Search query (search exit to close)**")
            query = input()
            print(f"{query}\n")
            if query == "exit":
                break

            print(f"**Select retrieval method**")
            print(f"1. Boolean Retrieval")
            print(f"2. Vector Space Model")
            print(f"3. Okapi BM25\n")
            method = int(input())

            if query:
                if method == 1:
                    results = boolean_retrieval(query, inverted_index)
                elif method == 2:
                    results = vsm_ranking(query, vectorizer, tfidf_matrix, threshold = 0.2)
                elif method == 3:
                    results = okapi_bm25(query, articles, threshold = 1.0)
                
                if len(results) > 0:
                    print(f"Found {len(results)} articles:")
                    for doc_id in results:
                        article = articles[doc_id]
                        print(f"[{article['title']}]({article['url']})")
                else:
                    print("No results found.")
        elif choice == 2:
            print(f"**Search Algorithm Evaluation**")
            evaluate_search_algorithms(articles, inverted_index, vectorizer, tfidf_matrix)
        elif choice == 3:
            print(f"**Exited**")
            break


Η συνάρτηση ***main*** παρέχει μια διαδραστική διεπαφή γραμμής εντολών για τους χρήστες, ώστε είτε να αναζητήσουν άρθρα από τη Wikipedia είτε να αξιολογήσουν την απόδοση των αλγορίθμων αναζήτησης (Boolean retrieval, Vector Space Model και Okapi BM25).

Αρχικά, φορτώνει τα άρθρα και τον αντιστραμμένο δείκτη, και υπολογίζει τον πίνακα TF-IDF για το Vector Space Model. Στη συνέχεια, εισέρχεται σε έναν βρόχο όπου ο χρήστης παρουσιάζεται με τρεις επιλογές:

**Αναζήτηση:** Ο χρήστης μπορεί να εισάγει ένα query αναζήτησης και να επιλέξει έναν από τους τρεις μεθόδους αναζήτησης (Boolean, VSM ή BM25). Ανάλογα με τη μέθοδο που επιλέγεται, η συνάρτηση ανακτά τα σχετικά άρθρα και τα εμφανίζει με τους τίτλους και τις διευθύνσεις URL τους. Εάν δεν βρεθούν αποτελέσματα, ενημερώνει τον χρήστη αναλόγως.

**Αξιολόγηση:** Αυτή η επιλογή αξιολογεί τους αλγορίθμους αναζήτησης με βάση τα προκαθορισμένα test queries και εκτυπώνει την ακρίβεια, την ανάκληση, το F1 score και το Mean Average Precision (MAP) για κάθε αλγόριθμο.

**Έξοδος:** Βγαίνει από τον βρόχο και τερματίζει το πρόγραμμα.

Η συνάρτηση τρέχει συνεχώς μέχρι ο χρήστης να επιλέξει την επιλογή εξόδου. Αυτή η διαδραστική ροή καθιστά εύκολο είτε να πραγματοποιήσετε αναζητήσεις είτε να αξιολογήσετε την αποτελεσματικότητα διαφορετικών αλγορίθμων αναζήτησης σε ένα μόνο μέρος.

Το παρακάτω κομμάτι κώδικα ελέγχει αν το αρχείο wikipedia_index.json υπάρχει ήδη. Εάν δεν υπάρχει, προχωρά με τα εξής βήματα:

**Ανακτά Άρθρα:** Καλεί τη συνάρτηση fetch_wikipedia_articles για να ανακτήσει άρθρα από τη Wikipedia σχετικά με το web scraping, περιορίζοντας τον αριθμό των άρθρων στα 50.

**Δημιουργεί Αντιστραμμένο Δείκτη:** Δημιουργεί έναν αντιστραμμένο δείκτη με βάση τα ανακτημένα άρθρα χρησιμοποιώντας τη συνάρτηση create_inverted_index.

**Αποθηκεύει Δεδομένα:** Τα άρθρα και ο αντιστραμμένος δείκτης αποθηκεύονται σε ένα αρχείο JSON (wikipedia_index.json) χρησιμοποιώντας τη συνάρτηση save_to_json. Αυτό εξασφαλίζει ότι τα δεδομένα είναι διαθέσιμα για μελλοντική χρήση χωρίς την ανάγκη να ανακτηθούν και να επεξεργαστούν ξανά.

Μετά από αυτό, εκτελείται η συνάρτηση main(), η οποία παρέχει στον χρήστη μια διαδραστική διεπαφή αναζήτησης.

Αυτή η προσέγγιση εξασφαλίζει ότι τα άρθρα ανακτώνται και επεξεργάζονται μόνο μία φορά, και οι επόμενες εκτελέσεις μπορούν να χρησιμοποιούν απευθείας τα προ-αποθηκευμένα δεδομένα, καθιστώντας τη διαδικασία πιο αποδοτική.

In [19]:
if not os.path.exists("wikipedia_index.json"):
    articles = fetch_wikipedia_articles("https://en.wikipedia.org/wiki/Web_scraping", max_articles = 50)
    inverted_index = create_inverted_index(articles)
    save_to_json({"articles": articles, "inverted_index": inverted_index})

main()

###Wikipedia Article Search Engine###

**Do you want to search a query or to evaluate the algorithms?**
1. Search
2. Evaluation
3. Exit

**Search query (search exit to close)**
data

**Select retrieval method**
1. Boolean Retrieval
2. Vector Space Model
3. Okapi BM25

Found 10 articles:
[Data extraction](https://en.wikipedia.org/wiki/Data_extraction)
[Data mining](https://en.wikipedia.org/wiki/Web_mining)
[Data mining](https://en.wikipedia.org/wiki/Data_mining)
[Web data integration](https://en.wikipedia.org/wiki/Web_data_integration)
[Data analysis](https://en.wikipedia.org/wiki/Data_analysis)
[Data scraping](https://en.wikipedia.org/wiki/Data_scraping)
[Data retrieval](https://en.wikipedia.org/wiki/Data_retrieval)
[Data feed](https://en.wikipedia.org/wiki/Data_feed)
[Database](https://en.wikipedia.org/wiki/Database)
[Semi-structured data](https://en.wikipedia.org/wiki/Semi-structured_data)

**Do you want to search a query or to evaluate the algorithms?**
1. Search
2. Evaluation
3. Ex